In [1]:
import os
import json
import shutil
import scrapbook as sb


/opt/conda/lib/python3.7/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


# Pipeline settings

In [2]:
src_dir = "../../src"  # Ignore this!

# Pipeline settings
task = "chinese_word_segmentation"  # chinese_word_segmentation OR target_classification, OR sequence_classification
model = None  # None: Default model 
device = 0
text_prepro = None  # None: Default steps 
model_params = {'num_train_epochs': 15}
train_params = {'batch_size': 16}
eval_params = {'batch_size': 32}
model_dir = "../output/test_pipeline_tmp"  # output dir for new model

In [2]:
os.chdir(src_dir)
from pipeline import Pipeline

NameError: name 'src_dir' is not defined

# Load data

In [3]:

train_raw_data = json.load(open(f"../data/datasets/sample/{task}/train_sample.json", 'r'))
dev_raw_data = json.load(open(f"../data/datasets/sample/{task}/train_sample.json", 'r'))
test_raw_data = json.load(open(f"../data/datasets/sample/{task}/train_sample.json", 'r'))
print(train_raw_data[0])

{'docid': 'S_00000235', 'content': '佢指，最大機會係有患者進入街市，經佢嘅糞便或者口水傳播，但究竟點傳播，係因為口水或者係由老鼠將病毒帶到四周圍，暫時未知道。', 'words': ['佢', '指', '，', '最', '大', '機會', '係', '有', '患者', '進入', '街市', '，', '經', '佢', '嘅', '糞便', '或者', '口水', '傳播', '，', '但', '究竟', '點', '傳播', '，', '係', '因為', '口水', '或者', '係', '由', '老鼠', '將', '病毒', '帶', '到', '四周圍', '，', '暫時', '未', '知道', '。'], 'postags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'sent_indexs': [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 7], [7, 8], [8, 9], [9, 11], [11, 13], [13, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 21], [21, 23], [23, 25], [25, 27], [27, 28], [28, 29], [29, 31], [31, 32], [32, 34], [34, 35], [35, 36], [36, 38], [38, 40], [40, 42], [42, 43], [43, 44], [44, 46], [46, 47], [47, 49], [49, 50], [50, 51], [51, 54], [54, 55], [55, 57], [57, 58], [58, 60], [60, 61]]}


# Run pipeline

## Initialize pipeline

In [4]:

pipeline = Pipeline(
    task=task, 
    model=model, 
    device=device, 
    text_prepro=text_prepro
)

2021-11-25 01:32:22 ***** Model class is not provided for chinese_word_segmentation. *****
2021-11-25 01:32:22   Default model = BERT_CRF


../config/examples/chinese_word_segmentation/BERT_CRF
['.ipynb_checkpoints', 'run.yaml', 'model', 'result', 'logs']


## Train a new model

In [5]:

pipeline.train(
    model_dir, 
    train_raw_data=train_raw_data, 
    dev_raw_data=dev_raw_data, 
    model_params=model_params,
    train_params=train_params
)

2021-11-25 01:32:24 ***** Initializing pipeline *****
2021-11-25 01:32:24 ***** Loading tokenizer *****
2021-11-25 01:32:24   Tokenizer source = 'transformers'
2021-11-25 01:32:26 ***** Initializing model *****
2021-11-25 01:32:26   Task = chinese_word_segmentation
2021-11-25 01:32:26   Model class = BERT_CRF


['tokenizer']


2021-11-25 01:32:27 ***** Loading pretrained language model *****
2021-11-25 01:32:27   Pretrained BERT = 'bert-base-chinese'
2021-11-25 01:32:39 ***** Loading data *****
2021-11-25 01:32:39   Raw data is provided.
32it [00:00, 227.13it/s]
2021-11-25 01:32:39   Loaded samples = 32
2021-11-25 01:32:39 ***** Loading data *****
2021-11-25 01:32:39   Raw data is provided.
32it [00:00, 172.57it/s]
2021-11-25 01:32:39   Loaded samples = 32
2021-11-25 01:32:39 ***** Running training *****
2021-11-25 01:32:39   Num examples = 32
2021-11-25 01:32:39   Num Epochs = 15
2021-11-25 01:32:39   Sampler = 
2021-11-25 01:32:39   Batch size = 16
2021-11-25 01:32:39   Gradient Accumulation steps = 1
Iteration: 100%|██████████| 2/2 [00:01<00:00,  1.57it/s, tr_loss=47.1]
2021-11-25 01:32:40 ***** Epoch end: 0 *****
2021-11-25 01:32:40 ***** Running evaluation *****
2021-11-25 01:32:40   Num examples = 32
2021-11-25 01:32:40   Batch size = 64

Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]
2021-1

## Test

In [6]:

metrics = pipeline.test(
    test_raw_data=test_raw_data,
    eval_params=eval_params
)

2021-11-25 01:16:31 ***** Loading data *****
2021-11-25 01:16:31   Raw data is provided.
32it [00:00, 211.66it/s]
2021-11-25 01:16:31   Loaded samples = 32
2021-11-25 01:16:31 ***** Running evaluation *****
2021-11-25 01:16:31   Num examples = 32
2021-11-25 01:16:31   Batch size = 32
Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]
2021-11-25 01:16:32   macro_f1 = 0.9983041266252121
2021-11-25 01:16:32   micro_f1 = 0.9983041266252121
2021-11-25 01:16:32   support = 884
2021-11-25 01:16:32   O-precision = 0.9977401129943503
2021-11-25 01:16:32   O-recall = 0.998868778280543
2021-11-25 01:16:32   O-f1-score = 0.9983041266252121
2021-11-25 01:16:32   O-support = 884
2021-11-25 01:16:32   loss = 0.4514961242675781
2021-11-25 01:16:32   dataset = train
2021-11-25 01:16:32 ***** Running evaluation *****
2021-11-25 01:16:32   Num examples = 32
2021-11-25 01:16:32   Batch size = 32
Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]
2021-11-25 01:16:33   macro_f1 = 0.99830412662

# Clear output folder

In [7]:
shutil.rmtree(model_dir)

# Export variables

In [8]:
sb.glue("macro_f1", metrics['macro_f1'])
sb.glue("micro_f1", metrics['micro_f1'])